## Segmenting and Clustering Neighborhoods
----------------------------------

### Step 1

In [64]:
#!conda install -y lxml
#!conda install -y  -c conda-forge geocoder 

import pandas as pd
import lxml
import geocoder
print(lxml.__file__)

/home/anaconda/juyter/6.0.3/lib/python3.8/site-packages/lxml/__init__.py


In [65]:
#Data URL
data_url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
print(data_url)

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M


In [100]:
#Read to panda dataframe
df = pd.read_html(data_url)
print(len(df))
df[0].keys()

3


Index(['Postal Code', 'Borough', 'Neighbourhood'], dtype='object')

In [101]:
#Get the first table from the wiki page and print the first ten rows
neighborhood_table=df[0]
#Change column names 
neighborhood_table.rename(columns={"Postal Code":"PostalCode"}, inplace=True)
neighborhood_table.head(3)


,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods


In [102]:
#Drop rows where Borough is Not assigned
neighborhood_table.drop(neighborhood_table[neighborhood_table.Borough=='Not assigned'].index, inplace=True)

In [103]:
#Check the new dataframe
print(len(neighborhood_table))
neighborhood_table.reset_index(drop=True,inplace=True)
neighborhood_table.head(3)

103


,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [70]:
neighborhood_table[neighborhood_table.Neighbourhood=='Not assigned'].index

Int64Index([], dtype='int64')

Quetions: If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
**There are no such row so not taking any actions**

In [71]:
neighborhood_table.shape

(103, 3)

------------------------------


### Step 2

In [18]:
import geocoder # import geocoder
g = geocoder.google('Toronto, Ontario')
print(g.latlng)
g

None


<[REQUEST_DENIED] Google - Geocode [empty]>

In [72]:
#Test geocoder 
#import geocoder # import geocoder

# initialize your variable to None
#lat_lng_coords = None

# loop until you get the coordinates
#while(lat_lng_coords is None):
#  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#  lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]
#print("latitude=",latitude)
#print("longitude=",longitude)

In [23]:
#goecoder fails, using the cvs
#coordinate_file="https://cocl.us/Geospatial_data"
#!wget -O 'coordinate.csv' coordinate_file https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
print('Data downloaded!')

Data downloaded!


In [84]:
coordinates = pd.read_csv("coordinate.csv") 
coordinates.head(3)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711


In [92]:
coordinates.rename(columns={"Postal Code":"PostalCode"}, inplace=True)
coordinates.set_index('PostalCode', inplace=True)
coordinates.head(10)

,Latitude,Longitude
PostalCode,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476
M1J,43.744734,-79.239476
M1K,43.727929,-79.262029
M1L,43.711112,-79.284577
M1M,43.716316,-79.239476


In [93]:
pc1="M1B"
c_long=coordinates.loc[pc1][0]
c_lati=coordinates.loc[pc1][1]
#print(coordinates.loc["M1B"][0])
print("Longitude=",c_long," Latitude=",c_lati)



Longitude= 43.806686299999996  Latitude= -79.19435340000001


In [106]:
#neighborhood_table.drop(columns=['Longitude='], inplace=True)
neighborhood_table['Longitude'] = 0
neighborhood_table['Latitude'] = 0
neighborhood_table.head(10)

,PostalCode,Borough,Neighbourhood,Longitude,Latitude
0,M3A,North York,Parkwoods,0,0
1,M4A,North York,Victoria Village,0,0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0
3,M6A,North York,"Lawrence Manor, Lawrence Heights",0,0
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",0,0
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",0,0
6,M1B,Scarborough,"Malvern, Rouge",0,0
7,M3B,North York,Don Mills,0,0
8,M4B,East York,"Parkview Hill, Woodbine Gardens",0,0
9,M5B,Downtown Toronto,"Garden District, Ryerson",0,0


In [117]:
neighborhood_table.loc[neighborhood_table['PostalCode'] == "M7A",'Longitude'] =1
neighborhood_table.head(10)

,PostalCode,Borough,Neighbourhood,Longitude,Latitude
0,M3A,North York,Parkwoods,1,0
1,M4A,North York,Victoria Village,0,0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0
3,M6A,North York,"Lawrence Manor, Lawrence Heights",0,0
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",1,0
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",0,0
6,M1B,Scarborough,"Malvern, Rouge",0,0
7,M3B,North York,Don Mills,0,0
8,M4B,East York,"Parkview Hill, Woodbine Gardens",0,0
9,M5B,Downtown Toronto,"Garden District, Ryerson",0,0


In [118]:
for pc in (neighborhood_table["PostalCode"]):
    c_long=coordinates.loc[pc][0]
    c_lati=coordinates.loc[pc][1]
    neighborhood_table.loc[neighborhood_table['PostalCode'] == pc,'Longitude'] = c_long
    neighborhood_table.loc[neighborhood_table['PostalCode'] == pc,'Latitude'] = c_lati
    #print(pc,"  Longitude=",c_long," Latitude=",c_lati)
    
neighborhood_table.head(10)

,PostalCode,Borough,Neighbourhood,Longitude,Latitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [122]:
tpc="M9V"
neighborhood_table.loc[neighborhood_table['PostalCode'] == tpc] 


,PostalCode,Borough,Neighbourhood,Longitude,Latitude
89,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
